# Pandas 3

---

## Content

- Introduction to IMDB use case
    - Merging `movies` & `directors` datasets
    - IMDB data exploration (Post-read)
- `apply()`
- `groupby()`
    - Group based Aggregation
    - Group based Filtering
    - Group based Apply

---

## **IMDB Movies Data**

- Imagine you are working as a Data Scientist for an analytics firm.
- Your task is to analyse some **movie trends** for a client.
- **IMDB** has an online database of information related to movies.

Let's download and read the IMDB dataset -

- File1: https://drive.google.com/file/d/1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd/view?usp=sharing
- File2: https://drive.google.com/file/d/1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm/view?usp=sharing


In [1]:
!pip install --upgrade gdown

In [2]:
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 70.9MB/s]


In [3]:
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm

Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 60.9MB/s]


Here we have two CSV files -
- `movies.csv`
- `directors.csv`

In [4]:
import pandas as pd
import numpy as np

In [5]:
movies = pd.read_csv('movies.csv')
movies.head()

,Unnamed: 0,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
4,5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


**So what kind of questions can we ask from this dataset?**

- **Top 10 most popular movies**, using `popularity`.
- Find the **highest rated movies**, using `vote_average`.
- We can find number of **movies released per year**.
- Find **highest budget movies in a year** using both `budget` and `year`.

**But can we ask more interesting/deeper questions?**

- Do you think we can find the **most productive directors**?
- Which **directors produce high budget films**?
- **Highest and lowest rated movies for every month** in a particular year?

Notice that there's a column **Unnamed: 0** which represents nothing but the index of a row.

**How to get rid of this `Unnamed: 0` col?**

In [6]:
movies = pd.read_csv('movies.csv', index_col=0)
movies.head()

,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


`index_col=0` explicitly states to treat the first column as the index.

The default value is `index_col=None`

In [7]:
movies.shape

(1465, 11)

The `movies` dataframe contains 1465 rows and 11 columns.

In [8]:
directors = pd.read_csv('directors.csv', index_col=0)
directors.head()

,director_name,id,gender
0,James Cameron,4762,Male
1,Gore Verbinski,4763,Male
2,Sam Mendes,4764,Male
3,Christopher Nolan,4765,Male
4,Andrew Stanton,4766,Male


In [9]:
directors.shape

(2349, 3)

The `directors` dataframe contains 2349 rows and 3 columns.

---

### Merging `movies` & `directors` datasets

**How can we know the details about the Director of a particular movie?**

- We will have to merge these two datasets.

**So on which column we should merge?**

We will use the **ID** columns (representing unique directors) in both the datasets.

If you observe,

- `director_id` of movies are taken from `id` of directors.
- Thus, we can merge our dataframes based on these two columns as **keys**.

Before that, let's first check the number of unique directors in our `movies` dataset.

**How do we get the number of unique directors in `movies`?**

In [10]:
movies['director_id'].nunique()

199

Recall, we had learnt about `nunique()` earlier.

In [11]:
directors['id'].nunique()

2349

**Summary:**

- `movies` dataset: 1465 rows, but only 199 unique directors
- `directors` dataset: 2349 unique directors (equal to the no. of rows)

**What can we infer from this?**

- The directors in `movies` data is a subset of directors in `directors` data.

**How can we check if all `director_id` values are present in `id`?**

In [12]:
movies['director_id'].isin(directors['id'])

,director_id
0,True
1,True
2,True
3,True
5,True
...,...
4736,True
4743,True
4748,True
4749,True


The `isin()` method checks if a column contains the specified value(s).

**How is `isin` different from Python's `in`?**

- `in` works for **one element** at a time.
- `isin` does this for **all the values** in the column.

If you notice,

- This is like a **boolean mask**.
- It returns a dataframe similar to the original one.
- For rows with values of `director_id` present in `id`, it returns True, else False.

**How can we check if there's any False here?**

In [13]:
np.all(movies['director_id'].isin(directors['id']))

np.True_

Let's finally merge the two dataframes.

Do we need to keep **all the rows for movies**? Yes!

Do we need to keep **all the rows of directors**? No.

- Only the ones for which we have a corresponding row in `movies`.

**So which `join` type do you think we should apply here?**

- `LEFT` Join

In [14]:
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day,director_name,id_y,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday,James Cameron,4762,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday,Gore Verbinski,4763,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday,Sam Mendes,4764,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday,Christopher Nolan,4765,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday,Sam Raimi,4767,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,4809,1978,May,Monday,Martin Scorsese,4809,Male
1461,48370,27000,19,3151130,Clerks,7.4,755,5369,1994,Sep,Tuesday,Kevin Smith,5369,Male
1462,48375,0,7,0,Rampage,6.0,131,5148,2009,Aug,Friday,Uwe Boll,5148,Male
1463,48376,0,3,0,Slacker,6.4,77,5535,1990,Jul,Friday,Richard Linklater,5535,Male


Notice the two strange id columns - `id_x` and `id_y`.

**What do you think these newly created columns are?**

Since the columns with name `id` are present in both the dataframes,
- `id_x` represents **id values from movie df**
- `id_y` represents **id values from directors df**

**Do you think any column is redundant here and can be dropped?**

- `id_y` is redundant as it is the same as `director_id`
- But we don't require the `director_id` any further.

So we can simply drop these features -

In [15]:
data.drop(['director_id','id_y'],axis=1,inplace=True)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


---

### Post-read

- [IMDB data exploration](https://colab.research.google.com/drive/1yrfHSQYUMxxLKGUG-gCPf-R232BuimiR?usp=sharing)

From here, we have the opportunity to delve into various aspects of the data, such as:

- Converting the revenue values into Millions of USD.
- Identifying the Top 5 most popular movies.

... and so on.

This task is for you to explore the data on your own.

Additionally, we've provided a notebook (accessible as **post-lecture content**) where this analysis has been conducted.

---

### `apply()`

- It is used apply a function along an axis of the DataFrame/Series.

Say we want to convert the data in `Gender` column into numerical format.

Basically,
- 0 for Male
- 1 for Female

**How can we encode the values in the `Gender` column?**

Let's first write a function to do it for a single value.

In [16]:
def encode(data):
  if data == "Male":
    return 0
  else:
    return 1

**Now how can we apply this function to the whole column?**

In [17]:
data['gender'] = data['gender'].apply(encode)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0


Notice how this is similar to using `Vectorization` in Numpy.

**How to apply a function on multiple columns?**

Let's say we want to find the sum of `revenue` and `budget` per movie?

In [18]:
data[['revenue', 'budget']].apply(np.sum)

,0
revenue,209866997305
budget,70353617179


We can pass multiple columns by packing them within `[]`.

But there's a mistake here. We wanted our results per movie (i.e. per row)

But we're getting the sum of the columns.

In [19]:
data[['revenue', 'budget']].apply(np.sum, axis=1)

,0
0,3024965087
1,1261000000
2,1125674609
3,1334939099
4,1148871626
...,...
1460,321952
1461,3178130
1462,0
1463,0


By setting the `axis=1`, every row of `revenue` was added to same row of `budget`.

**What does this `axis` mean in apply?**
- `axis=0` $\rightarrow$ It will apply to **each column**
- `axis=1` $\rightarrow$ It will apply to **each row**
  
Note that **by default, axis=0**.

**Similarly, how can I find the `profit` per movie (revenue-budget)?**

In [20]:
# We define a function to calculate profit

def prof(x):
  return x['revenue']-x['budget']
data['profit'] = data[['revenue', 'budget']].apply(prof, axis = 1)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0


---

### What is Grouping?

In simple terms, we could understood it through - Split, Apply, Combine

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912126/files/assets/pyds_03in01.png" height="350" width="700"/>

1. **Split**: Breaking up and grouping a DataFrame depending on the value of the specified key.

2. **Apply**: Computing some function, usually an aggregate, transformation, or filtering, within the individual groups.

3. **Combine**: Merging the results of these operations into an output array.

---

### Group based Aggregation

We use different aggregate functions like `mean`, `sum`, `min`, `max`, `count` etc. on columns while grouping.

Let's group our data director-wise.

In [21]:
data.groupby('director_name')

Notice,
- It's a **DataFrameGroupBy** type object
- **NOT a DataFrame** type object

**What's the number of groups our data is divided into?**

In [22]:
data.groupby('director_name').ngroups

199

Based on this grouping, we can find which keys belong to which group.

In [23]:
data.groupby('director_name').groups

{'Adam McKay': [176, 323, 366, 505, 839, 916], 'Adam Shankman': [265, 300, 350, 404, 458, 843, 999, 1231], 'Alejandro González Iñárritu': [106, 749, 1015, 1034, 1077, 1405], 'Alex Proyas': [95, 159, 514, 671, 873], 'Alexander Payne': [793, 1006, 1101, 1211, 1281], 'Andrew Adamson': [11, 43, 328, 501, 947], 'Andrew Niccol': [533, 603, 701, 722, 1439], 'Andrzej Bartkowiak': [349, 549, 754, 911, 924], 'Andy Fickman': [517, 681, 909, 926, 973, 1023], 'Andy Tennant': [314, 320, 464, 593, 676, 885], 'Ang Lee': [99, 134, 748, 840, 1089, 1110, 1132, 1184], 'Anne Fletcher': [610, 650, 736, 789, 1206], 'Antoine Fuqua': [310, 338, 424, 467, 576, 808, 818, 1105], 'Atom Egoyan': [946, 1128, 1164, 1194, 1347, 1416], 'Barry Levinson': [313, 319, 471, 594, 878, 898, 1013, 1037, 1082, 1143, 1185, 1345, 1378], 'Barry Sonnenfeld': [13, 48, 90, 205, 591, 778, 783], 'Ben Stiller': [209, 212, 547, 562, 850], 'Bill Condon': [102, 307, 902, 1233, 1381], 'Bobby Farrelly': [352, 356, 481, 498, 624, 630, 654, 80

**What if we want to extract data of a particular group from this list?**

In [24]:
data.groupby('director_name').get_group('Alexander Payne')

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
793,45163,30000000,19,105834556,About Schmidt,6.7,362,2002,Dec,Friday,Alexander Payne,1,75834556
1006,45699,20000000,40,177243185,The Descendants,6.7,934,2011,Sep,Friday,Alexander Payne,1,157243185
1101,46004,16000000,23,109502303,Sideways,6.9,478,2004,Oct,Friday,Alexander Payne,1,93502303
1211,46446,12000000,29,17654912,Nebraska,7.4,636,2013,Sep,Saturday,Alexander Payne,1,5654912
1281,46813,0,13,0,Election,6.7,270,1999,Apr,Friday,Alexander Payne,1,0


**How can we find the count of movies by each director?**

In [25]:
data.groupby('director_name')['title'].count()

,title
director_name,
Adam McKay,6
Adam Shankman,8
Alejandro González Iñárritu,6
Alex Proyas,5
Alexander Payne,5
...,...
Wes Craven,10
Wolfgang Petersen,7
Woody Allen,18


**How to find multiple aggregates of any feature?**

Finding the very first year and the latest year a director released a movie i.e basically the **min** & **max** of the `year` column, grouped by `director_name`.

In [26]:
data.groupby(['director_name'])["year"].aggregate(['min', 'max'])

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


**Note:** We can also use `.agg` instead of `.aggregate` (both are same)

---

### Group based Filtering

Group based filtering allows us to filter rows from each group by using conditional statements on each group rather than the whole dataframe.

**How to find the details of the movies by high budget directors?**

- Lets assume, high budget director -> any director with **atleast one movie with budget >100M**.

1. We can **group** the data by director and use `max` of the budget as aggregator.

In [58]:
data_dir_budget = data.groupby("director_name")["budget"].max().reset_index()
data_dir_budget

,director_name,budget
0,Adam McKay,100000000
1,Adam Shankman,80000000
2,Alejandro González Iñárritu,135000000
3,Alex Proyas,140000000
4,Alexander Payne,30000000
...,...,...
194,Wes Craven,40000000
195,Wolfgang Petersen,175000000
196,Woody Allen,30000000
197,Zack Snyder,250000000


2. We can **filter** out the director names with **max budget >100M**.

In [60]:
names = data_dir_budget.loc[data_dir_budget["budget"] >= 100000000, ["director_name",'budget']]

In [61]:
names

,director_name,budget
0,Adam McKay,100000000
2,Alejandro González Iñárritu,135000000
3,Alex Proyas,140000000
5,Andrew Adamson,225000000
10,Ang Lee,137000000
...,...,...
187,Tom Shadyac,175000000
188,Tom Tykwer,102000000
189,Tony Scott,100000000
195,Wolfgang Petersen,175000000


3. Finally, we can filter out the details of the movies by these directors.

In [67]:
data.loc[data['director_name'].isin(names['director_name'])]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1450,48267,400000,33,100000000,Mad Max,6.6,1213,1979,Apr,Thursday,George Miller,Male
1451,48268,200000,13,4505922,Swingers,6.8,253,1996,Oct,Friday,Doug Liman,Male
1452,48274,0,5,2611555,Three,6.3,31,2010,Dec,Thursday,Tom Tykwer,Male
1458,48335,60000,27,3221152,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male


**Can we filter groups in a single go using Lambda functions?** Yes!

In [69]:
data.groupby('director_name').filter(lambda x: x["budget"].max() >= 100000000)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1450,48267,400000,33,100000000,Mad Max,6.6,1213,1979,Apr,Thursday,George Miller,Male
1451,48268,200000,13,4505922,Swingers,6.8,253,1996,Oct,Friday,Doug Liman,Male
1452,48274,0,5,2611555,Three,6.3,31,2010,Dec,Thursday,Tom Tykwer,Male
1458,48335,60000,27,3221152,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male


Notice what's happening here?

- We first group data by director and then use `groupby().filter` function.
- **Groups are filtered if they do not satisfy the boolean criterion** specified by the function.
- This is called **Group Based Filtering**.

**Note:**
- We are filtering the **groups** here and **not the rows**.
- The result is **not a groupby object** but regular **Pandas DataFrame** with the **filtered groups eliminated**.

---

### Group based Apply

- applying a function on grouped objects

**What if we want to do the transformation of a column using some column's agrregate**

Let's say, we want to filter the risky movies whose budget was even higher than the average revenue of the director from his other movies.

We can subtract the average `revenue` of a director from `budget` column, for each director.

In [31]:
def func(x):
  # returns whether a movie is risky or not
  x["risky"] = x["budget"] - x["revenue"].mean() >= 0
  return x

data_risky = data.groupby("director_name", group_keys=False).apply(func)
data_risky

/tmp/ipython-input-4275592603.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_risky = data.groupby("director_name", group_keys=False).apply(func)


,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550965087,False
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661000000,False
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635674609,False
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834939099,False
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632871626,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,321952,False
1461,48370,27000,19,3151130,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3124130,False
1462,48375,0,7,0,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0,False
1463,48376,0,3,0,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0,False


**Note:**
- Setting `group_keys=True`, keeps the group key in the returned dataset.
- This will be default in future versions of Pandas.
- Keep it as False if want the normal behaviour.

**What did we do here?**

- Defined a custom function.
- Grouped data according to `director_name`.
- Subtracted the mean of `budget` from `revenue`.
- Used apply with the custom function on the grouped data.

Now let's see if there are any risky movies -

In [32]:
data_risky.loc[data_risky["risky"]]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit,risky
7,43608,200000000,107,586090727,Quantum of Solace,6.1,2965,2008,Oct,Thursday,Marc Forster,0,386090727,True
12,43614,380000000,135,1045713802,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,0,665713802,True
15,43618,200000000,37,310669540,Robin Hood,6.2,1398,2010,May,Wednesday,Ridley Scott,0,110669540,True
20,43624,209000000,64,303025485,Battleship,5.5,2114,2012,Apr,Wednesday,Peter Berg,0,94025485,True
24,43630,210000000,3,459359555,X-Men: The Last Stand,6.3,3525,2006,May,Wednesday,Brett Ratner,0,249359555,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,47224,5000000,7,3263585,The Sweet Hereafter,6.8,103,1997,May,Wednesday,Atom Egoyan,0,-1736415,True
1349,47229,5000000,3,4842699,90 Minutes in Heaven,5.4,40,2015,Sep,Friday,Michael Polish,0,-157301,True
1351,47233,5000000,6,0,Light Sleeper,5.7,15,1992,Aug,Friday,Paul Schrader,1,-5000000,True
1356,47263,15000000,10,0,Dying of the Light,4.5,118,2014,Dec,Thursday,Paul Schrader,1,-15000000,True


---

### Multi-Indexing

In [33]:
!pip install --upgrade gdown

In [34]:
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 64.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 91.5MB/s]


In [35]:
movies = pd.read_csv('movies.csv', index_col=0)
directors = pd.read_csv('directors.csv', index_col=0)

data = movies.merge(directors, how='left', left_on='director_id', right_on='id')
data.drop(['director_id','id_y'], axis=1, inplace=True)

**Which director according to you should be considered as most productive?**

- Should we decide based on the **number of movies** directed?
- Or take the **quality of the movies** into consideration as well?
- Or maybe look at the the **amount of business** the movie is doing?

To simplify, let's calculate who has directed maximum number of movies.

In [36]:
data.groupby(['director_name'])['title'].count().sort_values(ascending=False)

,title
director_name,
Steven Spielberg,26
Martin Scorsese,19
Clint Eastwood,19
Woody Allen,18
Robert Rodriguez,16
...,...
Stephen Daldry,5
Tom Tykwer,5
Tim Hill,5


`Steven Spielberg` has directed maximum number of movies.

**But does it make `Steven` the most productive director?**

- Chances are, he might be active for more years than the other directors.

**Calculating the active years for every director?**

- We can subtract both `min` and `max` of year.

In [37]:
data_agg = data.groupby(['director_name'])[["year", "title"]].aggregate({"year":['min','max'], "title": "count"})
data_agg

year       title
                              min   max count
director_name                                
Adam McKay                   2004  2015     6
Adam Shankman                2001  2012     8
Alejandro González Iñárritu  2000  2015     6
Alex Proyas                  1994  2016     5
Alexander Payne              1999  2013     5
...                           ...   ...   ...
Wes Craven                   1984  2011    10
Wolfgang Petersen            1981  2006     7
Woody Allen                  1977  2013    18
Zack Snyder                  2004  2016     7
Zhang Yimou                  2002  2014     6

[199 rows x 3 columns]

Notice,
- `director_name` column has turned into **row labels**.
- There are multiple levels for the column names.

This is called a **Multi-index DataFrame**.

- It can have **multiple indexes along a dimension**.
  - The no. of dimensions remain same though.
- Multi-level indexes are **possible both for rows and columns**.

In [38]:
data_agg.columns

MultiIndex([( 'year',   'min'),
            ( 'year',   'max'),
            ('title', 'count')],
           )

The level-1 column names are `year` and `title`.

**What would happen if we print the column `year` of this multi-index dataframe?**

In [39]:
data_agg["year"]

,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


**How can we convert multi-level back to only one level of columns?**

- e.g. `year_min`, `year_max`, `title_count`

In [40]:
data_agg = data.groupby(['director_name'])[["year","title"]].aggregate(
    {"year":['min', 'max'], "title": "count"})

In [41]:
data_agg.columns = ['_'.join(col) for col in data_agg.columns]
data_agg

,year_min,year_max,title_count
director_name,,,
Adam McKay,2004,2015,6
Adam Shankman,2001,2012,8
Alejandro González Iñárritu,2000,2015,6
Alex Proyas,1994,2016,5
Alexander Payne,1999,2013,5
...,...,...,...
Wes Craven,1984,2011,10
Wolfgang Petersen,1981,2006,7
Woody Allen,1977,2013,18


Since these were tuples, we can just join them.

In [42]:
data.groupby('director_name')[['year', 'title']].aggregate(
    year_max=('year','max'),
    year_min=('year','min'),
    title_count=('title','count')
)

,year_max,year_min,title_count
director_name,,,
Adam McKay,2015,2004,6
Adam Shankman,2012,2001,8
Alejandro González Iñárritu,2015,2000,6
Alex Proyas,2016,1994,5
Alexander Payne,2013,1999,5
...,...,...,...
Wes Craven,2011,1984,10
Wolfgang Petersen,2006,1981,7
Woody Allen,2013,1977,18


The columns look good, but we may want to turn back the row labels into a proper column as well.

**Converting row labels into a column using `reset_index` -**

In [43]:
data_agg.reset_index()

,director_name,year_min,year_max,title_count
0,Adam McKay,2004,2015,6
1,Adam Shankman,2001,2012,8
2,Alejandro González Iñárritu,2000,2015,6
3,Alex Proyas,1994,2016,5
4,Alexander Payne,1999,2013,5
...,...,...,...,...
194,Wes Craven,1984,2011,10
195,Wolfgang Petersen,1981,2006,7
196,Woody Allen,1977,2013,18
197,Zack Snyder,2004,2016,7


**Using the new features, can we find the most productive director?**

1. First calculate how many years the director has been active.

In [44]:
data_agg["yrs_active"] = data_agg["year_max"] - data_agg["year_min"]
data_agg

,year_min,year_max,title_count,yrs_active
director_name,,,,
Adam McKay,2004,2015,6,11
Adam Shankman,2001,2012,8,11
Alejandro González Iñárritu,2000,2015,6,15
Alex Proyas,1994,2016,5,22
Alexander Payne,1999,2013,5,14
...,...,...,...,...
Wes Craven,1984,2011,10,27
Wolfgang Petersen,1981,2006,7,25
Woody Allen,1977,2013,18,36


2. Then calculate rate of directing movies by `title_count`/`yrs_active`.

In [45]:
data_agg["movie_per_yr"] = data_agg["title_count"] / data_agg["yrs_active"]
data_agg

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Adam McKay,2004,2015,6,11,0.545455
Adam Shankman,2001,2012,8,11,0.727273
Alejandro González Iñárritu,2000,2015,6,15,0.400000
Alex Proyas,1994,2016,5,22,0.227273
Alexander Payne,1999,2013,5,14,0.357143
...,...,...,...,...,...
Wes Craven,1984,2011,10,27,0.370370
Wolfgang Petersen,1981,2006,7,25,0.280000
Woody Allen,1977,2013,18,36,0.500000


3. Finally, sort the values.

In [46]:
data_agg.sort_values("movie_per_yr", ascending=False)

,year_min,year_max,title_count,yrs_active,movie_per_yr
director_name,,,,,
Tyler Perry,2006,2013,9,7,1.285714
Jason Friedberg,2006,2010,5,4,1.250000
Shawn Levy,2002,2014,11,12,0.916667
Robert Rodriguez,1992,2014,16,22,0.727273
Adam Shankman,2001,2012,8,11,0.727273
...,...,...,...,...,...
Lawrence Kasdan,1985,2012,5,27,0.185185
Luc Besson,1985,2014,5,29,0.172414
Michael Apted,1980,2010,5,30,0.166667


**Conclusion:**

- `Tyler Perry` turns out to be truly the most productive director.

---